In [14]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, TextLoader
from PyPDF2 import PdfReader
from langchain.vectorstores import Pinecone
import pinecone

from config import config

In [15]:
file_path = 'data/inputs/sample.pdf'

In [16]:
reader = PdfReader(file_path)
text = ''
for page in reader.pages:
    text += page.extract_text()

with open(f'data/outputs/output.txt', 'w') as file:
    file.write(text)

In [17]:
loader = DirectoryLoader(
    'data/outputs/',
    glob='**/*.txt',
    loader_cls=TextLoader
)

In [18]:
documents = loader.load()

In [19]:
text_splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=1024,
    chunk_overlap=0
)

In [20]:
texts = text_splitter.split_documents(documents)

In [21]:
embeddings = OpenAIEmbeddings(
    openai_api_key=config.OPENAI_API_KEY
)

In [24]:
pinecone.init(
    api_key=config.PINECONE_API_KEY,
    environment=config.PINECONE_ENVIRONMENT,
)

In [31]:
index_name = pinecone.Index('test')

In [34]:
vectordb = Pinecone.from_documents(
    documents=texts,
    embedding=embeddings,
    index_name='test'
)